In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Spoiler Detection/LM-BFF"
%ls

/content/drive/MyDrive/Spoiler Detection/LM-BFF
auto_label_mapping/        log.lock
auto_template/             new_env/
average_logits.py          old_ensemble_predict_results/
copy_ensemble_predicts.sh  README.md
count_tokens.ipynb         requirements.txt
count_tokens.py            result/
data/                      run_experiment.sh
discord_log                run.ipynb
eda.ipynb                  run_multiple.sh
ensemble_predict_results/  run.py
ensemble_predict.sh        spoilers_auto_label_mapping/
env/                       spoilers_auto_template/
evaluate.ipynb             src/
evaluate.sh                template_search.sh
figs/                      tmp.txt
label_search.sh            tools/
LICENSE                    train_individual.sh
log                        uncertain_log


In [ ]:

!source env/bin/activate; which python

/usr/local/bin/python


In [ ]:
!apt-get update
!apt-get install python3.8-dev python3.8-venv
!source env/bin/activate; python -m pip install --upgrade pip

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,343 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,269 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-securit

In [ ]:
!find . -type f -name '*.lock' -delete

^C


In [ ]:
!nvidia-smi

Wed Oct 11 03:36:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Divide unlabeled set (test.csv) into chunks of 260000
!source env/bin/activate; TAG="pred_v1.0_batch32" TYPE=prompt TASK=spoilers BS=32 LR=1e-5 SEED=21 MODEL=roberta-large bash evaluate.sh \
 "--template *cls**sent_0*._It_was*mask*.*sep+* --mapping {0:'relevant',1:'irrelevant'} --first_sent_limit 502 --other_sent_limit 502"

01/23/2023 01:18:46 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
01/23/2023 01:18:46 - INFO - __main__ -   Training/evaluation parameters DynamicTrainingArguments(output_dir='result/pred_v1.0_batch32', overwrite_output_dir=False, do_train=False, do_eval=False, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=128, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=16, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=0.0, max_steps=4500, warmup_steps=0, logging_dir='runs/Jan23_01-18-46_b3dd4534b7aa', logging_first_step=False, logging_steps=225, save_steps=500, save_total_limit=None, no_cuda=False, seed=21, fp16=False, fp16_opt_level

In [ ]:
# !rm "data/unlabeled_v1.0.1/test.csv"
# !rm "data/unlabeled_v1.0.1/chunk_*""
!rm "data/unlabeled_v1.0.1/cached_*"

rm: cannot remove '/content/drive/MyDrive/Spoiler Detection/spoiler-detection/data/unlabeled_v1.0.1/cached_\*': No such file or directory


In [ ]:
!cat ensemble_predict.sh

#!/bin/bash
for template_id in {49..50}
do
    # for seed in 13 21 42 87 100
    for seed in 21
    do
        # To save time, we fix these hyper-parameters
        bs=32
        lr=1e-5
        
        TAG=irrelevant-relevant-template-ensemble-predict
        TYPE=prompt
        TASK=spoilers
        BS=$bs
        LR=$lr
        SEED=$seed
        MODEL=roberta-large
        TEMPLATE_ID=$template_id
    
        K=3615
        NUM_EPOCHS=20
        # Training steps
        MAX_STEP=$(($K * 2 / $BS * $NUM_EPOCHS))
        
        # Validation steps
        EVAL_STEP=$(($K * 2 / $BS * 1))
        LOG_STEP=$(($EVAL_STEP / 10))
        
        TASK_EXTRA=""

        MAPPING="{0:'relevant',1:'irrelevant'}"
        
        REAL_BS=2
        GS=$(expr $BS / $REAL_BS)
        
        DATA_DIR="data/unlabeled_v1.0.1"
        
        CHECKPOINT=result/$template_id/
        
        # Since we only use dev performance here, use --no_predict to skip testing
        python run.py \
        

In [ ]:
# Divide unlabeled set (test.csv) into chunks of 260000
!source env/bin/activate; bash ensemble_predict.sh

10/11/2023 04:07:26 - INFO - __main__ -   Load top-100 templates from spoilers_auto_template/irrelevant_relevant/16-21.txt
10/11/2023 04:07:26 - INFO - __main__ -   Specify load the 49-th template: *cls**sent_0*▁at*mask*.*sep+*
10/11/2023 04:07:26 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/11/2023 04:07:26 - INFO - __main__ -   Training/evaluation parameters DynamicTrainingArguments(output_dir='result/autolabeled_49', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=128, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=16, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_ep

In [ ]:
!source env/bin/activate; TAG="stuffed_v1.0_batch32" TYPE=prompt TASK=spoilers BS=32 LR=1e-5 SEED=21 MODEL=roberta-large bash run_experiment.sh \
 "--template *cls**sent_0*._It_was*mask*.*sep+* --mapping {0:'relevant',1:'irrelevant'} --first_sent_limit 502 --other_sent_limit 502"

/content/drive/MyDrive/Spoiler Detection/LM-BFF/env/lib/python3.8/site-packages/transformers/training_args.py:337: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  warnings.warn(
08/29/2023 07:19:48 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
08/29/2023 07:19:48 - INFO - __main__ -   Training/evaluation parameters DynamicTrainingArguments(output_dir='result/stuffed_v1.0_batch32', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=True, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=16, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsil

In [ ]:
!source env/bin/activate; TAG="v1.0_demo" TYPE=prompt-demo TASK=spoilers BS=32 LR=1e-5 SEED=21 MODEL=roberta-large bash run_experiment.sh \
 "--template *cls**sent_0*._It_was*mask*.*sep+* --mapping {0:'relevant',1:'irrelevant'} --first_sent_limit 160 --other_sent_limit 160"

Streaming output truncated to the last 5000 lines.
 15% 410/2698 [11:10<1:02:40,  1.64s/it]
 15% 411/2698 [11:12<1:02:43,  1.65s/it]
 15% 412/2698 [11:14<1:02:29,  1.64s/it]
 15% 413/2698 [11:15<1:02:27,  1.64s/it]
 15% 414/2698 [11:17<1:02:33,  1.64s/it]
 15% 415/2698 [11:19<1:02:27,  1.64s/it]
 15% 416/2698 [11:20<1:02:27,  1.64s/it]
 15% 417/2698 [11:22<1:02:31,  1.64s/it]
 15% 418/2698 [11:24<1:02:24,  1.64s/it]
 16% 419/2698 [11:25<1:02:24,  1.64s/it]
 16% 420/2698 [11:27<1:02:30,  1.65s/it]
 16% 421/2698 [11:29<1:02:14,  1.64s/it]
 16% 422/2698 [11:30<1:02:14,  1.64s/it]
 16% 423/2698 [11:32<1:02:08,  1.64s/it]
 16% 424/2698 [11:33<1:02:06,  1.64s/it]
 16% 425/2698 [11:35<1:02:12,  1.64s/it]
 16% 426/2698 [11:37<1:02:11,  1.64s/it]
 16% 427/2698 [11:38<1:02:10,  1.64s/it]
 16% 428/2698 [11:40<1:02:17,  1.65s/it]
 16% 429/2698 [11:42<1:02:09,  1.64s/it]
 16% 430/2698 [11:43<1:02:02,  1.64s/it]
 16% 431/2698 [11:45<1:02:05,  1.64s/it]
 16% 432/2698 [11:47<1:02:09,  1.65s/it]
 16% 4

In [ ]:
!source env/bin/activate; TAG="v1.0_FT_batch32" TYPE=finetune TASK=spoilers BS=32 LR=1e-5 SEED=21 MODEL=roberta-large bash run_experiment.sh

/content/drive/MyDrive/Spoiler Detection/LM-BFF/env/lib/python3.7/site-packages/transformers/training_args.py:339: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
11/07/2022 12:20:26 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/07/2022 12:20:26 - INFO - __main__ -   Training/evaluation parameters DynamicTrainingArguments(output_dir='result/v1.0_FT_batch32', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=True, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=16, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e

In [ ]:
!source env/bin/activate; TAG="v1.0_FT_batch32" TYPE=finetune TASK=spoilers BS=32 LR=1e-5 SEED=21 MODEL=roberta-large bash evaluate.sh

11/08/2022 06:22:36 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/08/2022 06:22:36 - INFO - __main__ -   Training/evaluation parameters DynamicTrainingArguments(output_dir='result/v1.0_FT_batch32', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=16, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=0.0, max_steps=4500, warmup_steps=0, logging_dir='runs/Nov08_06-22-36_199c3484481b', logging_first_step=False, logging_steps=225, save_steps=500, save_total_limit=None, no_cuda=False, seed=21, fp16=False, fp16_opt_level='O1

In [ ]:
# relevant, prompt-demo: 87.2
# relevant, prompt: 87.3
# this_is_mask, prompt: 88.2
# LR: 1e-05, BS: 4
!source env/bin/activate; python tools/gather_result.py --condition "{'tag': 'it_was_mask', 'task_name': 'spoilers', 'few_shot_type': 'prompt', 'per_device_train_batch_size': 4}"

13-13: best dev (0.88959) test (0.88205) test2 (0.73179) | total trials: 1
    | gradient_accumulation_steps: 1 | learning_rate: 1e-05 | per_device_train_batch_size: 4 | eval_steps: 100 | max_steps: 1000 
21-21: best dev (0.86528) test (0.86851) test2 (0.76821) | total trials: 1
    | gradient_accumulation_steps: 1 | learning_rate: 1e-05 | per_device_train_batch_size: 4 | eval_steps: 100 | max_steps: 1000 
42-42: best dev (0.90591) test (0.86804) test2 (0.83444) | total trials: 1
    | gradient_accumulation_steps: 1 | learning_rate: 1e-05 | per_device_train_batch_size: 4 | eval_steps: 100 | max_steps: 1000 
87-87: best dev (0.87574) test (0.87654) test2 (0.76490) | total trials: 1
    | gradient_accumulation_steps: 1 | learning_rate: 1e-05 | per_device_train_batch_size: 4 | eval_steps: 100 | max_steps: 1000 
100-100: best dev (0.88034) test (0.87764) test2 (0.76490) | total trials: 1
    | gradient_accumulation_steps: 1 | learning_rate: 1e-05 | per_device_train_batch_size: 4 | eval_ste

In [ ]:
!source env/bin/activate; python tools/get_sbert_embedding.py --do_test --sbert_model roberta-large --k 3615 --data_dir "data/k-shot/spoilers/binary_v1.0/3615-21" --seed 21 --task spoilers

100% 1.31G/1.31G [01:33<00:00, 14.1MB/s]
spoilers-3615-21-train
0it [00:00, ?it/s]| [Excitement intensifies]
7230it [03:12, 37.58it/s]
spoilers-3615-21-dev
0it [00:00, ?it/s]| Welp I now have confirmation that they won’t be uploading it to Hulu this season. Motherfuckers. Time to dust of the tricorne and head for my ship.
2698it [01:11, 37.57it/s]
spoilers-3615-21-test
0it [00:00, ?it/s]| Finally watched it. This episode was way better. I loved Jeans inner struggle to choose between the world and himself/future family. Is that Mikasa he was imagining being married to? I saw a scar on her cheek. Hange was kinda weird. She claims she isn't their superior and then says shes still the commander of the survey corps. She points that all the dead soldiers wouldnt agree with genocide but what about the yeagerist. Many of them were once cadets or in the new survey corps and they agreed with the rumbling why would all the dead soldiers who only knew that humanity lived within the walls fight for

In [ ]:
!source env/bin/activate; python tools/generate_template.py \
    --output_dir spoilers_auto_template \
    --task_name spoilers \
    --seed 21 \
    --t5_model t5-3b \
    --beam 100

Downloading: 100% 1.20k/1.20k [00:00<00:00, 882kB/s]
Downloading: 100% 11.4G/11.4G [06:22<00:00, 29.8MB/s]
Downloading: 100% 792k/792k [00:00<00:00, 1.86MB/s]
# spoilers 16 21 100
| dataset examples
| {'label': 1, 'text': ['Bye bye levi']}
| {'label': 0, 'text': ['HE MADE THE OST?']}

| mapping
| {0: 'relevant', 1: 'irrelevant'}
####### example #######
Bye bye levi<extra_id_0> irrelevant<extra_id_1>
I like how they added this lil transformation effect on the hange-eren scene<extra_id_0> irrelevant<extra_id_1>
I have a big feeling they'll use youseebiggirl in Warhammer's transformation reveal next ep<extra_id_0> irrelevant<extra_id_1>
####### example #######

100% 18/18 [06:55<00:00, 23.08s/it]
####### generated results #######
--------------
score: -2.8529539108276367
generated ids [32099, 6, 32098, 5, 1]
generated text <extra_id_0>,<extra_id_1>.</s>
--------------
score: -2.979557991027832
generated ids [32099, 11, 32098, 5, 1]
generated text <extra_id_0>▁and<extra_id_1>.</s>
--------

In [ ]:
!source env/bin/activate; bash template_search.sh

/content/drive/MyDrive/Spoiler Detection/LM-BFF/env/lib/python3.8/site-packages/transformers/training_args.py:337: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  warnings.warn(
08/28/2023 23:31:55 - INFO - __main__ -   Load top-100 templates from spoilers_auto_template/irrelevant_relevant/16-21.txt
08/28/2023 23:31:55 - INFO - __main__ -   Specify load the 80-th template: *cls*▁Also*mask*.*+sent_0**sep+*
08/28/2023 23:31:55 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
08/28/2023 23:31:55 - INFO - __main__ -   Training/evaluation parameters DynamicTrainingArguments(output_dir='result/80', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=True, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_si

In [ ]:
!cat template_search.sh

In [ ]:
!source env/bin/activate; python tools/sort_template.py --condition "{'tag': 'irrelevant-relevant-template-v5', 'task_name': 'spoilers'}" --template_dir spoilers_auto_template --name irrelevant_relevant

Seed 21 has 36 results


In [ ]:
!python average_logits.py --logits_dir ensemble_predict_results/prompt/test --sorted_templates spoilers_auto_template/irrelevant_relevant/16-21.score.txt --n_models 20

Ensembling template spoilers-47-test.npy
Ensembling template spoilers-48-test.npy
Ensembling template spoilers-49-test.npy
Ensembling template spoilers-50-test.npy
Ensembling template spoilers-53-test.npy
Ensembling template spoilers-54-test.npy
Ensembling template spoilers-56-test.npy
Ensembling template spoilers-57-test.npy
Ensembling template spoilers-58-test.npy
Ensembling template spoilers-59-test.npy
Ensembling template spoilers-60-test.npy
Ensembling template spoilers-64-test.npy
Ensembling template spoilers-65-test.npy
Ensembling template spoilers-66-test.npy
Ensembling template spoilers-67-test.npy
Ensembling template spoilers-70-test.npy
Ensembling template spoilers-71-test.npy
Ensembling template spoilers-72-test.npy
Ensembling template spoilers-73-test.npy
Ensembling template spoilers-79-test.npy


In [ ]:
!cat average_logits.py

In [ ]:
# !source env/bin/activate; bash tools/run_generate_labels.sh

+ K=16
+ DATA_DIR=data/k-shot-10x
+ OUTPUT_DIR=spoilers_auto_label_mapping
+ MODEL_NAME=roberta-large
+ LOAD_TEMPLATES=true
+ TEMPLATE_DIR=spoilers_auto_template
+ TEMPLATE_NAME=yes_no
+ NUM_TEMPLATES=20
+ K_LIKELY=100
+ K_NEIGHBORS=30
+ N_PAIRS=100
+ TASKS=spoilers
+ SEEDS=21
+ TASK_EXTRA=
+ for TASK in $TASKS
+ for SEED in $SEEDS
+ case $TASK in
+ TEMPLATE='*cls**sent_0*._Spoiler?*mask*.*sep+*'
+ MAPPING='{0:'\''No'\'',1:'\''Yes'\''}'
+ [[ true = \t\r\u\e ]]
+ FILENAME=spoilers_auto_template/yes_no/16-21.sort.txt
++ head -n 20 spoilers_auto_template/yes_no/16-21.sort.txt
+ for TEMPLATE in $(head -n $NUM_TEMPLATES $FILENAME)
+ python tools/generate_labels.py --overwrite_output_dir --output_dir /tmp/output --model_name_or_path roberta-large --output_file spoilers_auto_label_mapping/auto_template/spoilers/16-21.txt --append_output_file --write_template --template '*cls**sent_0*.*mask*s.*sep+*' --mapping '{0:'\''No'\'',1:'\''Yes'\''}' --task_name spoilers --data_dir data/k-shot-10x/spoil

In [ ]:
# !source env/bin/activate; bash label_search.sh

Streaming output truncated to the last 5000 lines.
 50% 6/12 [00:01<00:01,  4.92it/s]
 58% 7/12 [00:01<00:01,  4.73it/s]
 67% 8/12 [00:01<00:00,  4.60it/s]
 75% 9/12 [00:01<00:00,  4.51it/s]
 83% 10/12 [00:02<00:00,  4.45it/s]
 92% 11/12 [00:02<00:00,  4.42it/s]
100% 12/12 [00:02<00:00,  5.09it/s]03/08/2022 05:11:42 - INFO - src.trainer -   Best dev result: 0.9383749961853027
Epoch:  20% 49/250 [01:32<05:35,  1.67s/it]
13it [00:52, 15.14s/it]            
14it [00:52, 10.67s/it]
15it [00:52,  7.54s/it]
16it [00:53,  5.34s/it]
17it [00:53,  3.81s/it]
18it [00:53,  2.74s/it]
19it [00:53,  1.98s/it]
20it [00:54,  1.46s/it]
21it [00:54,  1.09s/it]
22it [00:54,  1.20it/s]
23it [00:54,  1.53it/s]
24it [00:54,  2.02it/s]03/08/2022 05:12:34 - INFO - src.trainer -   Best dev result: 0.9388749599456787
Epoch:  30% 74/250 [02:23<04:52,  1.66s/it]
25it [01:43, 15.03s/it]
26it [01:44, 10.59s/it]
27it [01:44,  7.48s/it]
28it [01:44,  5.31s/it]
29it [01:44,  3.78s/it]
30it [01:44,  2.72s/it]
31it [01:

In [ ]:
!source env/bin/activate; bash ensemble.sh

Streaming output truncated to the last 5000 lines.
2224it [15:36,  4.33it/s]
2225it [15:36,  4.33it/s]
2226it [15:36,  4.33it/s]
2227it [15:36,  4.33it/s]
2228it [15:37,  4.33it/s]
2229it [15:37,  4.33it/s]
2230it [15:37,  4.33it/s]
2231it [15:37,  4.33it/s]
2232it [15:38,  4.33it/s]
2233it [15:38,  4.33it/s]
2234it [15:38,  4.33it/s]
2235it [15:38,  4.33it/s]
2236it [15:39,  4.33it/s]
2237it [15:39,  4.33it/s]
2238it [15:39,  4.33it/s]
2239it [15:39,  4.33it/s]
2240it [15:39,  4.33it/s]
2241it [15:40,  4.33it/s]
2242it [15:40,  4.33it/s]
2243it [15:40,  4.33it/s]
2244it [15:40,  4.33it/s]
2245it [15:41,  4.33it/s]
2246it [15:41,  4.33it/s]
2247it [15:41,  4.33it/s]
2248it [15:41,  4.33it/s]
2249it [15:42,  4.33it/s]
2250it [15:42,  4.33it/s]
2251it [15:42,  4.33it/s]
2252it [15:42,  4.33it/s]
2253it [15:42,  4.33it/s]
2254it [15:43,  4.33it/s]
2255it [15:43,  4.33it/s]
2256it [15:43,  4.33it/s]
2257it [15:43,  4.33it/s]
2258it [15:44,  4.33it/s]
2259it [15:44,  4.33it/s]
2260it [15:44

In [ ]:
!source env/bin/activate; python tools/ensemble.py --condition "{'tag': 'yes_no_ensemble', 'task_name': 'spoilers', 'few_shot_type': 'prompt-demo'}" --n_models 20

Model 0 | val: mean +- std: 94.4 +- 0.0 | test: mean +- std: 88.4 (0.0) (median 88.4) / 95.8 +- 0.0 (median 95.8)
Model 1 | val: mean +- std: 94.4 +- 0.0 | test: mean +- std: 90.3 (0.0) (median 90.3) / 95.8 +- 0.0 (median 95.8)
Model 2 | val: mean +- std: 95.2 +- 0.0 | test: mean +- std: 89.7 (0.0) (median 89.7) / 95.8 +- 0.0 (median 95.8)
Model 3 | val: mean +- std: 88.0 +- 0.0 | test: mean +- std: 83.4 (0.0) (median 83.4) / 91.7 +- 0.0 (median 91.7)
Model 4 | val: mean +- std: 91.6 +- 0.0 | test: mean +- std: 85.1 (0.0) (median 85.1) / 95.8 +- 0.0 (median 95.8)
Model 5 | val: mean +- std: 86.5 +- 0.0 | test: mean +- std: 86.7 (0.0) (median 86.7) / 75.0 +- 0.0 (median 75.0)
Model 6 | val: mean +- std: 87.7 +- 0.0 | test: mean +- std: 84.4 (0.0) (median 84.4) / 100.0 +- 0.0 (median 100.0)
Model 7 | val: mean +- std: 93.1 +- 0.0 | test: mean +- std: 90.2 (0.0) (median 90.2) / 83.3 +- 0.0 (median 83.3)
Model 8 | val: mean +- std: 86.0 +- 0.0 | test: mean +- std: 81.8 (0.0) (median 81.8) 